In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import datetime, gc, random
import json, numpy as np, pandas as pd, zipfile
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.cluster import MeanShift, estimate_bandwidth
# file
import zipfile

# Data preprocessing
import datetime
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import roc_curve, roc_auc_score, auc
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVR
import warnings
warnings.filterwarnings('ignore', message='A column-vector y was passed when a 1d array was expected.')

In [10]:
#數據前處理
def processing_df(df):
    df["Type"] = df["Type"].map({"ScalaSparkKmeans":1, "ScalaSparkNWeight":2,"ScalaSparkPagerank":3,"ScalaSparkTerasort":4,"ScalaSparkWordcount":5})
    df["spark.serializer"] = df["spark.serializer"].map({"org.apache.spark.serializer.JavaSerializer": 1, "org.apache.spark.serializer.KryoSerializer": 2})
    df["spark.io.compression.codec"] = df["spark.io.compression.codec"].map({"snappy":1, "lz4":2,"lzf":3})
    #欄位為 str的轉 int , Ex: "26k" -> 26
    df['spark.broadcast.blockSize'] = df['spark.broadcast.blockSize'].str.replace('m','').astype(int)
    df['spark.driver.memory'] = df['spark.driver.memory'].str.replace('g','').astype(int)
    df['spark.executor.memory'] = df['spark.executor.memory'].str.replace('g','').astype(int)
    df['spark.io.compression.lz4.blockSize'] = df['spark.io.compression.lz4.blockSize'].str.replace('k','').astype(int)
    df['spark.io.compression.snappy.blockSize'] = df['spark.io.compression.snappy.blockSize'].str.replace('k','').astype(int)
    df['spark.kryoserializer.buffer'] = df['spark.kryoserializer.buffer'].str.replace('k','').astype(int)
    df['spark.kryoserializer.buffer.max'] = df['spark.kryoserializer.buffer.max'].str.replace('m','').astype(int)
    df['spark.reducer.maxSizeInFlight'] = df['spark.reducer.maxSizeInFlight'].str.replace('m','').astype(int)
    df['spark.shuffle.file.buffer'] = df['spark.shuffle.file.buffer'].str.replace('k','').astype(int)
    df['spark.storage.memoryMapThreshold'] = df['spark.storage.memoryMapThreshold'].str.replace('m','').astype(int)
    return df

def error_index(true_data,pred_data,sample_type):
    df_true = pd.DataFrame(true_data)
    df_true = df_true.reset_index()
    df_pred = pd.DataFrame(pred_data,columns = ['Duration(s)_pred'])
    df_all = df_true.merge(df_pred, how='inner', left_index=True, right_index=True)
    df_all["error_ratio"] = (abs((df_all['Duration(s)_pred'] - df_all['Duration(s)'])) / df_all['Duration(s)']*100)
    df_all["mse"] = (df_all['Duration(s)'] - df_all['Duration(s)_pred']) ** 2
    #df_all.to_csv("rf_report_"+sample_type+".csv")
    
    return df_all["error_ratio"].mean()

type = 'ScalaSparkKmeans'
hibench_report_simple = pd.read_csv('/home/love1245672/桌面/hibench_report/dataset_simple/'+ type +'.csv')
hibench_report_grid = pd.read_csv('/home/love1245672/桌面/hibench_report/dataset_grid/'+ type +'.csv')

hibench_report_simple = processing_df(hibench_report_simple)
hibench_report_grid = processing_df(hibench_report_grid)

In [11]:
banchmark = ["ScalaSparkKmeans","ScalaSparkNWeight","ScalaSparkPagerank","ScalaSparkTerasort","ScalaSparkWordcount"]
for ratio in range(1,2):
    result = pd.DataFrame()
    for type in banchmark:
        hibench_report_simple = pd.read_csv('/home/love1245672/桌面/hibench_report/dataset_simple/'+ type +'.csv')
        hibench_report_grid = pd.read_csv('/home/love1245672/桌面/hibench_report/dataset_grid/'+ type +'.csv')

        hibench_report_simple = processing_df(hibench_report_simple)
        hibench_report_grid = processing_df(hibench_report_grid)

        for size in range(50,750,50):
            print(size)
            mean_mse = 0
            mean_error = 0
            mean_mse_grid = 0
            mean_error_grid = 0
            mean_mse_mix = 0
            mean_error_mix = 0
            for i in range(20):
                
                # 從 DataFrame 中隨機抽取 700 列數據作為訓練集
                train_data_simple = hibench_report_simple.sample(n=size, random_state=i) 
                train_data_grid = hibench_report_grid.sample(n=size, random_state=i)
                train_data_mix = pd.concat([train_data_simple.sample(n=size*2//10, random_state=i), train_data_grid.sample(n=size*(8)//10, random_state=i)])
                
                X_train_simple = train_data_simple[['spark.driver.memory','spark.driver.cores','spark.executor.instances','spark.reducer.maxSizeInFlight','spark.shuffle.file.buffer','spark.shuffle.sort.bypassMergeThreshold','spark.memory.fraction','spark.memory.storageFraction','spark.shuffle.memoryFraction', 'spark.storage.memoryFraction','spark.storage.unrollFraction', 'spark.default.parallelism','spark.broadcast.blockSize', 'spark.storage.memoryMapThreshold','spark.io.compression.codec', 'spark.io.compression.lz4.blockSize','spark.io.compression.snappy.blockSize', 'spark.kryoserializer.buffer.max','spark.kryoserializer.buffer','spark.serializer']]
                y_train_simple = train_data_simple[['Duration(s)']]
                
                X_train_grid = train_data_grid[['spark.driver.memory','spark.driver.cores','spark.executor.instances','spark.reducer.maxSizeInFlight','spark.shuffle.file.buffer','spark.shuffle.sort.bypassMergeThreshold','spark.memory.fraction','spark.memory.storageFraction','spark.shuffle.memoryFraction', 'spark.storage.memoryFraction','spark.storage.unrollFraction', 'spark.default.parallelism','spark.broadcast.blockSize', 'spark.storage.memoryMapThreshold','spark.io.compression.codec', 'spark.io.compression.lz4.blockSize','spark.io.compression.snappy.blockSize', 'spark.kryoserializer.buffer.max','spark.kryoserializer.buffer','spark.serializer']]
                y_train_grid = train_data_grid[['Duration(s)']]
                
                X_train_mix = train_data_mix[['spark.driver.memory','spark.driver.cores','spark.executor.instances','spark.reducer.maxSizeInFlight','spark.shuffle.file.buffer','spark.shuffle.sort.bypassMergeThreshold','spark.memory.fraction','spark.memory.storageFraction','spark.shuffle.memoryFraction', 'spark.storage.memoryFraction','spark.storage.unrollFraction', 'spark.default.parallelism','spark.broadcast.blockSize', 'spark.storage.memoryMapThreshold','spark.io.compression.codec', 'spark.io.compression.lz4.blockSize','spark.io.compression.snappy.blockSize', 'spark.kryoserializer.buffer.max','spark.kryoserializer.buffer','spark.serializer']]
                y_train_mix = train_data_mix[['Duration(s)']]
                
                # 從剩餘的數據中選擇作為測試集
                test_data_simple = hibench_report_simple[~hibench_report_simple.index.isin(train_data_simple.index)].sample(n=50, random_state=i)
                test_data_grid = hibench_report_grid[~hibench_report_grid.index.isin(train_data_grid.index)].sample(n=50, random_state=i)
                test_data = pd.concat([test_data_simple, test_data_grid])

                X_test = test_data[['spark.driver.memory','spark.driver.cores','spark.executor.instances','spark.reducer.maxSizeInFlight','spark.shuffle.file.buffer','spark.shuffle.sort.bypassMergeThreshold','spark.memory.fraction','spark.memory.storageFraction','spark.shuffle.memoryFraction', 'spark.storage.memoryFraction','spark.storage.unrollFraction', 'spark.default.parallelism','spark.broadcast.blockSize', 'spark.storage.memoryMapThreshold','spark.io.compression.codec', 'spark.io.compression.lz4.blockSize','spark.io.compression.snappy.blockSize', 'spark.kryoserializer.buffer.max','spark.kryoserializer.buffer','spark.serializer']]
                y_test = test_data[['Duration(s)']]

                forest_simple = RandomForestRegressor()
                forest_grid = RandomForestRegressor()
                forest_mix = RandomForestRegressor()

                # Fitting
                forest_simple = forest_simple.fit(X_train_simple, y_train_simple)
                forest_grid = forest_grid.fit(X_train_grid, y_train_grid)
                forest_mix = forest_mix.fit(X_train_mix, y_train_mix)

                mean_mse += mean_squared_error(y_test, forest_simple.predict(X_test))
                mean_error += error_index(y_test, forest_simple.predict(X_test),"simple")

                mean_mse_grid += mean_squared_error(y_test, forest_grid.predict(X_test))
                mean_error_grid += error_index(y_test, forest_grid.predict(X_test),"grid")

                mean_mse_mix += mean_squared_error(y_test, forest_mix.predict(X_test))
                mean_error_mix += error_index(y_test, forest_mix.predict(X_test),"mix")

            #print("Mean_MSE_simple:{}".format(mean_mse/20))
            #print("Mean_Error_simple:{}".format(mean_error/20))
            #print("")
            #print("Mean_MSE_grid:{}".format(mean_mse_grid/20))
            #print("Mean_Error_grid:{}".format(mean_error_grid/20))
            #print("")
            #print("Mean_MSE_mix:{}".format(mean_mse_mix/20))
            #print("Mean_Error_mix:{}".format(mean_error_mix/20))
            #print("----------------------------------------------")
            result = result.append({"size": size, "simple_"+type[10:]+"_error": mean_error/20,"grid_"+type[10:]+"_error": mean_error_grid/20,"mix_"+type[10:]+"_error": mean_error_mix/20},ignore_index=True)
    result.to_csv("result_"+str(ratio)+str(10-ratio)+".csv")


450


/tmp/ipykernel_5115/3940924290.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"size": size, "simple_"+type[10:]+"_error": mean_error/20,"grid_"+type[10:]+"_error": mean_error_grid/20,"mix_"+type[10:]+"_error": mean_error_mix/20},ignore_index=True)


450


/tmp/ipykernel_5115/3940924290.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"size": size, "simple_"+type[10:]+"_error": mean_error/20,"grid_"+type[10:]+"_error": mean_error_grid/20,"mix_"+type[10:]+"_error": mean_error_mix/20},ignore_index=True)


450


...

/tmp/ipykernel_5115/3940924290.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"size": size, "simple_"+type[10:]+"_error": mean_error/20,"grid_"+type[10:]+"_error": mean_error_grid/20,"mix_"+type[10:]+"_error": mean_error_mix/20},ignore_index=True)


450


/tmp/ipykernel_5115/3940924290.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"size": size, "simple_"+type[10:]+"_error": mean_error/20,"grid_"+type[10:]+"_error": mean_error_grid/20,"mix_"+type[10:]+"_error": mean_error_mix/20},ignore_index=True)


450


/tmp/ipykernel_5115/3940924290.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({"size": size, "simple_"+type[10:]+"_error": mean_error/20,"grid_"+type[10:]+"_error": mean_error_grid/20,"mix_"+type[10:]+"_error": mean_error_mix/20},ignore_index=True)


In [ ]:
print(train_data_grid.shape)
print(train_data_simple.shape)
print(train_data_mix.shape)

(700, 28)
(700, 28)
(700, 28)
